In [2]:
# Obtiene una máscara para cada municipio del país.

!pip install bottleneck

import os

import pandas as pd
import numpy as np

import xarray as xr

from multiprocessing import Pool

In [3]:
# Obtiene el promedio espacual por municipio.
def proc_mean(y):
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[k] + var + "/mexico_" 
        + var + "." + str(y) + "_monthly.nc" ) as ds:
        # Establecemos np.nan como valor sin datos.
        ds = ds.where(ds > -9000)
        if k == 0:
            df_y = df[ df["Año"] == y ].copy()
        else: df_y = df.copy()
        for i in df["CVE_INEGI"]:
            # Obtenemos el promedio dentro
            # de la máscara del municipio.
            df_y.loc[ df["CVE_INEGI"] == i,
                var ] = ( ds.where(mask
                .sel(CVE_INEGI = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe()
                .drop(columns = "CVE_INEGI")
                .values )
        return df_y

# Obtiene el promedio temporal y opera con los valores mensuales
def proc_mean_month(y):
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[k] + var + "/mexico_" 
        + var + "." + str(y) + "_monthly.nc" ) as ds:
        # Establecemos np.nan como valor sin datos.
        ds = ds.where(ds > -9000)
        if k == 0:
            df_y = df[ df["Año"] == y ].copy()
        else: df_y = df.copy()
        for i in df["CVE_INEGI"]:
            # Obtenemos el promedio dentro
            # de la máscara del municipio.
            df_m = ( ds.where(mask
                .sel(CVE_INEGI = i) == 1 )
                .mean(dim = ["lat", "lon"])
                .to_dataframe()
                .drop(columns = "CVE_INEGI")
                .iloc[:, 0] )
            # Ordenamos los datos de mayor a menor.
            df_m_1 = df_m.sort_values(ascending = False)
            # Ordenamos la suma de 6 meses
            # (verano) de mayor a menor
            df_m_2 = ( df_m.loc[::-1]
              .rolling(6).mean()
              .sort_values(ascending = False) )
            # Segundo mes más alto.
            df_y.loc[ df["CVE_INEGI"] == i,
                "Tmean_max_2" ] = df_m_1.iloc[1]
            # Mes de inicio de la temporada más calida.
            df_y.loc[ df["CVE_INEGI"] == i,
                "M_verano" ] = df_m_2.index[0]
            # Temperatura media.
            df_y.loc[ df["CVE_INEGI"] == i,
                var ] = df_m.values
        return df_y
  
# Obtiene la suma temporal y espacial.
def proc_soc(y):
    # Abrimos el netCDF del año y variable correspondiente.
    with xr.open_dataset( path_d[k] + var + "/mexico_" 
        + var + "." + str(y) + "_yearly.nc" ) as ds:
        df_y = df.copy()
        for i in df["CVE_INEGI"]:
            # Obtenemos el promedio dentro
            #  de la máscara del municipio.
            df_y.loc[ df["CVE_INEGI"] == i,
                var ] = ( ds.where(mask
                .sel(CVE_INEGI = i) == 1 )
                .sum("lat").to_dataframe()
                .drop(columns = "CVE_INEGI")
                .sum()[0] )
        return df_y

In [4]:
path = "/content/drive/MyDrive/Colab/Subsidio_electricidad/" 

# Escenarios y años.
RCP = ["RCP4p5", "RCP8p5"]

# Directorios.
dirs = ["Actual"] + RCP
path_data = [path + "data/" + x + "/" for x in dirs]
folder = "vars/"
path_d = [path + folder + x + "/" for x in dirs]


# Variables para generar la máscara.
vars = ["Tmax", "Tmin", "Tmean", "Pre",
        "HDD_mean", "CDD_mean", "HDD_p10",
        "CDD_p90", "Pre_Tmean", "Poblacion", "PIB"]

# Nombre de los archivos.
f_mask = "/Municipios/municipios_mask.nc"
csv = ".csv"

# Máscara para los municipios.
mask_0 = xr.open_dataset( path + f_mask )

In [5]:
# Procesa los datos futuros.

fut = [2030, 2050, 2070]

# Variable a procesar, 0 - 10.
n = 8
var = vars[n]

mask = mask_0.copy()

if var in vars[4:8]:
    mask = mask.rename_vars(mask = "degree_day")
else:
    mask = mask.rename_vars(mask = var)
    
# Para el PIB y la población,
# se agrega un año actual.
if n in [9, 10]:
    fut = [2010] + fut
    
# Iteramos en los RCPs.
for k in range(len(path_d[1:])):
    k += 1
        
    # Iteramos en los años.
    for f in fut:
        b = 1

        # Abrimos los archivos a utilizar.
        df = pd.read_csv( path_data[k] +
            "data_0." + str(f) + csv )
                    
        df_r = []

        # Población  PIB
        if var in vars[9:11]:
            # Por el momento, solo para RCP8.5
            if k != 1:
                fut = [2010] + fut
                df_r = proc_soc(f)
                # Elimina el respaldo, si existe.
                if os.path.exists( path_data[k] +
                    "data_" + str(n + 1) + "." +
                    str(f) + ".original.csv" ):
                    os.remove( path_data[k] +
                    "data_" + str(n + 1) + "." +
                    str(f) + ".original.csv" )

                # Crea un dato por cada mes.
                df_r = df_r.loc[ df_r.index.repeat(12) ]
                df_r["Mes"] = list(range(1, 13)) * 2469
                df_r.reset_index( drop = True , inplace = True )
            else: b = 0
        # Calcula además la temperatura mínima
        # máxima y el mes de inicio del verano.
        elif var == "Tmean":
            # Crea un dato por cada mes.
            df = df.loc[df.index.repeat(12)]
            df["Mes"] = list(range(1, 13)) * 2469

            df_r = proc_mean_month(f)
        # Variables que se promedian.
        else:
            # Crea un dato por cada mes.
            df = df.loc[df.index.repeat(12)]
            df["Mes"] = list(range(1, 13)) * 2469

            df_r = proc_mean(f)

        if b:
            # Guardamos el resultado como un nuevo archivo.
            df_r.to_csv(path_data[k] + "data_" + str(n + 1)
                + "." + str(f) + ".csv", index = False)
            print(path_data[k] + "data_" + str(n + 1)
                + "." + str(f) + ".csv")

/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP4p5/data_9.2030.csv
/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP4p5/data_9.2050.csv
/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP4p5/data_9.2070.csv
/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data_9.2030.csv
/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data_9.2050.csv
/content/drive/MyDrive/Colab/Subsidio_electricidad/data/RCP8p5/data_9.2070.csv


In [ ]:
# Procesa los datos actuales.

# Variable a procesar, 0 - 8
n = 0
p = 1
var = vars[n]

mask = mask_0.copy()

if var in vars[4:8]:
    mask = mask.rename_vars(mask = "degree_day")
else:
    mask = mask.rename_vars(mask = var)

# Datos actuales 
k = 0

if p == 0:
    # Abrimos los archivos a utilizar.
    df = pd.read_csv( path_data[k]
        + "data_0" + csv)
    
    # Crea un dato por cada mes.
    df = df.loc[df.index.repeat(12)]
    df["Mes"] = list(range(1, 13)) * 2469 * 7
    df.reset_index(drop = True, inplace = True)
    
    years = list(range(2010, 2014))

else:
    # Abrimos los archivos a utilizar.  
    df = pd.read_csv( path_data[k]
        + "data_" + str( n + 1 ) + csv )

    years = list(range(2014, 2017))

df_r = []

# Procesamos los años en paralelo.
pool = Pool()
if var == "Tmean":
    df_r = pool.map(proc_mean_month, years)
else:
    # Variables que se promedian.
    df_r = pool.map(proc_mean, years)

# Concatenamos los resultados de cada año.
for p in df_r:
    df.loc[df["Año"] == p["Año"].max() ] = p

# Guardamos el resultado como un nuevo archivo.
df.to_csv(path_data[k] + "data_" +
    str(n + 1) + csv, index = False)
print(path_data[k] + "data_" +
    str(n + 1) + csv)